In [1]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
print(tf.config.list_physical_devices('GPU'))
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.4.0
2.4.3
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

In [4]:
directory = "data/"
stock_data_list = []

for filename in os.listdir(directory):
  file_path = os.path.join(directory, filename)
  stock_data = pd.read_csv(file_path)
  stock_data = stock_data.rename(columns={'Data': 'Date', 'Otwarcie': 'Open', 'Najwyzszy': 'Highest', 'Najnizszy': 'Lowest', 'Zamkniecie': 'Close', 'Wolumen': 'Volume'})
  stock_data['Date'] = pd.to_datetime(stock_data.Date)
  stock_data_list.append(stock_data)
  print('Loaded file: ' + filename)

for i in range(len(stock_data_list)):
  stock_data_list[i] = stock_data_list[i].dropna()

Loaded file: acp_d.csv
Loaded file: ale_d.csv
Loaded file: alr_d.csv
Loaded file: ccc_d.csv
Loaded file: cdr_d.csv
Loaded file: cps_d.csv
Loaded file: dnp_d.csv
Loaded file: jsw_d.csv
Loaded file: kgh_d.csv
Loaded file: lpp_d.csv
Loaded file: lts_d.csv
Loaded file: opl_d.csv
Loaded file: peo_d.csv
Loaded file: pge_d.csv
Loaded file: pgn_d.csv
Loaded file: pkn_d.csv
Loaded file: pko_d.csv
Loaded file: pzu_d.csv
Loaded file: san_d.csv
Loaded file: tpe_d.csv


In [ ]:
def get_moving_average(data, period):
    data['MA'] = data['Close'].rolling(period).mean()
    return data

In [ ]:
def get_RSI (data, period):
    increase_difference, decrease_difference = data['Close'].diff(), data['Close'].diff()
    increase_difference[increase_difference < 0] = 0
    decrease_difference[decrease_difference > 0] = 0
    roll_increase = increase_difference.ewm(span = period).mean()
    roll_decrease = decrease_difference.abs().ewm(span = period).mean()
    RS = roll_increase / roll_decrease
    data['RSI'] = 100 - (100 / (1 + RS))

In [ ]:
def get_ROC(data):
    data['ROC'] = data['Close'].pct_change()
    return(data)

In [ ]:
def get_stochastic_oscillator(data, period):
    L14, H14 = data['Close'].rolling(period).min(), data['Close'].rolling(period).max()
    data['K'] = (data['Close'] - L14)/(H14 - L14)
    return(data)

In [ ]:
def get_SO_moving_average(data, period):
    L14, H14 = data['Close'].rolling(so_period).min(), data['Close'].rolling(so_period).max()
    K = (data['Close'] - L14)/(H14 - L14)

  Moving average of the Stochastic Oscillator D
  data['D'] = data['K'].rolling(so_d_period).mean()
    return(data)

In [ ]:
def calculate_technical_indicators(data, rsi_period=10, so_period=14, so_d_period=4, tema_period=10, cgi_period=20, wpi_period=14):
  # rsi_period - number of sessions considered when calculating RSI
  # so_period - number of sessions considered when calculating stochastic oscillator K
  # so_d_period - numbers of sessions considered when calculating moving average of the stochastic oscillator K
  # tema_period - number of sessions considered when calculating TEMA
  # cgi_period - number of sessions considered when calculating CGI
  # wpi_period - number of sessions considered when calculating Williams' Percent Range

  # Moving averages for periods of 10, 30 and 60 days
#   data['MovingAverage4'] = data['Close'].rolling(4).mean()
#   data['MovingAverage7'] = data['Close'].rolling(7).mean()
  data['MovingAverage20'] = data['Close'].rolling(20).mean()

  # Relative Strength Index RSI
  increase_difference, decrease_difference = data['Close'].diff(), data['Close'].diff()
  increase_difference[increase_difference < 0] = 0
  decrease_difference[decrease_difference > 0] = 0
  roll_increase = increase_difference.ewm(span = rsi_period).mean()
  roll_decrease = decrease_difference.abs().ewm(span = rsi_period).mean()
  RS = roll_increase / roll_decrease
#  data['RSI'] = 100 - (100 / (1 + RS))

  # Rate of Change ROC
#  data['ROC'] = data['Close'].pct_change()

  # Stochastic Oscillator K
  L14, H14 = data['Close'].rolling(so_period).min(), data['Close'].rolling(so_period).max()
#  data['K'] = (data['Close'] - L14)/(H14 - L14)

  # Moving average of the Stochastic Oscillator D
#   data['D'] = data['K'].rolling(so_d_period).mean()

  # Moving Average Convergence / Divergence MACD
  EMA_26 = data['Close'].ewm(26).mean()
  EMA_12 = data['Close'].ewm(12).mean()
#   data['MACD'] = EMA_12 - EMA_26

  # MACD Signal Line
#   data['MACD_Signal'] = data['MACD'].ewm(9).mean()

  # MACD histogram
#   data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

  # Percentage Price Oscillator PPO
#  data['PPO'] =(EMA_12 - EMA_26)/EMA_26

  # Triple Exponential Moving Average TEMA
  SEMA = data['Close'].ewm(tema_period).mean()
  DEMA = SEMA.ewm(tema_period).mean()
#   data['TEMA'] = DEMA.ewm(tema_period).mean()

  # Commodity Channel Index CGI
  typical_price = (data['Highest'] + data['Lowest'] + data['Close']) / 3
  MA = typical_price.rolling(cgi_period).mean()
  mean_deviation = (MA - typical_price).abs().rolling(cgi_period).mean()
#  data['CCI'] = (typical_price - MA) / (0.15 * mean_deviation)

  # Williams' Percent Range
#  data['Percent_Range'] = (data['Highest'].rolling(wpi_period).max() - data['Close']) / (data['Highest'].rolling(wpi_period).max() - data['Lowest'].rolling(wpi_period).min())

  return data